<a href="https://colab.research.google.com/github/kikiru328/Bone_Detection/blob/main/Final_Auto_code_BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Source Code
공용폴더에 폴더(yolov5), 파일(model.pt) 업로드해뒀습니다.

## Preprocessing Code

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import glob
import math


def read_img(path):
    original_img = cv2.imread(path)
    return original_img

########## Making mask for removing background #############

def make_mask(original_img):

    ## change to lab for making mask

    img_mask = original_img.copy()

    img_mask = cv2.cvtColor(img_mask, cv2.COLOR_RGB2BGR)
    img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2Lab)
    
    ## blur _02 
    # kernel_size = odds / value = img.mean()


    blur_k = int((img_mask.mean()*0.5)//2)*2+1
    img_mask = cv2.medianBlur(img_mask, blur_k)
    
    ## change to Grayscale for threshold

    img_mask = cv2.cvtColor(img_mask, cv2.COLOR_Lab2BGR)
    img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)

    ## binary / value = img.mean()

    if img_mask.mean() > 100 : 
      th = img_mask.mean()*0.94
    else : 
      th = img_mask.mean()

    ret, img_mask = cv2.threshold(img_mask, th, 255, cv2.THRESH_BINARY)

    ## mask based Max value of contours

    contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key=cv2.contourArea)
    mask = np.zeros(img_mask.shape, dtype=np.uint8)
    cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)
    
    ## Applying for dilation

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (8,8))
    mask = cv2.dilate(mask,k)
    return mask

######## background cut based mask ##########

def cut_mask(original_img, mask):

    ## copying
    img_for_cut = original_img.copy()

    ## H/W
    height, width = img_for_cut.shape[:2]

    ## mask
    mask_list = mask.tolist()
    
    for y in range(int(height*0.05),height):
        if max(mask[y,int(width*0.3):int(width*0.7)]) > 0:
            start_y = y-int(height*0.05)
            break
            
    for x in range(int(width*0.05),width):
        if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
            start_x = x-int(width*0.05)
            break
            
    for x in range(int(width*0.95),-1,-1):
        if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
            end_x = x+int(width*0.05)
            break
            
    cut_index = 0
    if mask_list[height-1][-1] == 255 or mask_list[height-1][0] == 255:
        for n in reversed(range(height)):
            if mask_list[n][0] == 0 or mask_list[n][-1] == 0:
                cut_index = n
                break
                
    if cut_index == 0:
        cut_index = height

    ## converting color
    img_for_cut = cv2.cvtColor(img_for_cut, cv2.COLOR_BGR2GRAY) 

    img_for_cut = img_for_cut[start_y:(cut_index-1),start_x:end_x]
    mask = mask[start_y:(cut_index-1),start_x:end_x]

    ## remove background
    masked = cv2.bitwise_and(img_for_cut, mask)

    return masked


######## Overlaying ########

def overlay_contours(masked):
    copied_img = masked.copy()

    ## smoothing
    gray_smooth = cv2.bilateralFilter(copied_img, 7,60,60)

    ## contours
    outline = cv2.Canny(gray_smooth,50,50)

    ## overlay
    overlaied_img = cv2.add(masked,outline)
    return overlaied_img

######## Rotation ########

def img_rotation(overlaied_img):

    ## copying img
    before_rot_img = overlaied_img.copy()

    
    h, w = before_rot_img.shape[:2]
    before_rot_img = cv2.cvtColor(before_rot_img, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(before_rot_img, cv2.COLOR_BGR2GRAY)
    ret, th = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    th_li = th.tolist()

    ## Rotation stage 01
    # lower = first black spot

    for i in reversed(range(h)):
        if th_li[i][0] == 0 and th_li[i][-1] == 0:
            lower = i
            break

    # lower = condition ; bottom = lower / img * 0.95

    if lower == h - 1:
        lower = int(h*0.9)

    # upper = condition ; lower + lower * 0.05

    slice5 = int(len(th)*0.05)
    upper = lower - slice5

    # x, y = between upper and lower (5%) / wrist center

    x,y = [],[]
    for i in range(slice5):
        cnt = th_li[i + upper].count(255)
        index = th_li[i + upper].index(255)
        x.append([i+upper])
        y.append([int((index*2 + cnt - 1)/2)])

    # x, y / draw regression line

    model = LinearRegression()
    model.fit(X=x,y=y)

    ####################################################

    ## Rotation stage 02
    
    angle = math.atan2(h - 0, int(model.predict([[h]])) - int(model.predict([[0]])))*180/math.pi
    M = cv2.getRotationMatrix2D((w/2,h/2), angle-90, 1)
    rotate = cv2.warpAffine(before_rot_img, M, (w, h))

    # Cutting img (rotated img)

    for i in range(len(th[-1])):
        if th[-1][i] == 255:
            start_x = i
            break

    for i in range(len(th[-1])):
        if th[-1][i] == 255:
            end_x = i
            

    s_point = h - int((int(model.predict([[h]])-start_x)) * math.tan(math.pi*((90-angle)/180)))
    e_point = h - int((end_x - int(model.predict([[h]]))) * math.tan(math.pi*((angle-90)/180)))
    point = max(s_point, e_point)
    rotated_img = rotate[:point]
    return rotated_img

######## Decompose_contrast ########
def contrast_roi(img, low, high):

    ## height, width

    h, w = img.shape
    img_ = np.zeros(img.shape, dtype=np.uint8)

    for y in range(h):
        for x in range(w):
            temp = int((255 / (high - low)) * (img[y][x] - low))
            if temp > 255:
                img_[y][x] = 255
            elif temp < 0:
                img_[y][x] = 0
            else:
                img_[y][x] = temp
    return img_

######## Decompose_brightness ########
def bright_ness(img):
    ## columns, rows
    cols, rows = img.shape[:2]
    brightness = np.sum(img) / (255 * cols * rows)
    return brightness

######## Decomposing ########

### img, morphology_value_1, morphology_value_2, filter_value(a,b)

def Decomposing(rotated_img,a,b,d,e):

    ######## Decomposing_stage_1 / [ Contours , Mask ] ########
    decomp_img_1 = rotated_img.copy()

    ## Adjusting brighness

    if bright_ness(decomp_img_1) > 0.8:
        decomp_img_1 = np.clip(decomp_img_1 - 80., 0, 255).astype(np.uint8)
    elif bright_ness(decomp_img_1) > 0.75:
        decomp_img_1 = np.clip(decomp_img_1 - 50., 0, 255).astype(np.uint8)
    elif bright_ness(decomp_img_1) > 0.65:
        decomp_img_1 = np.clip(decomp_img_1 - 30., 0, 255).astype(np.uint8)
    else: decomp_img_1 = np.clip(decomp_img_1 - 10., 0, 255).astype(np.uint8)


    ## change to Lab

    decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_RGB2BGR)
    decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_BGR2Lab)

    ## Morphology

    k = cv2.getStructuringElement(cv2.MORPH_CROSS, (a, a))
    decomp_img_1 = cv2.morphologyEx(decomp_img_1, cv2.MORPH_TOPHAT, k) # Emphasis

    ## Filter

    decomp_img_1 = cv2.bilateralFilter(decomp_img_1,-1, d, e)

    ## Lab to gray for binary

    decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_Lab2BGR)
    decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_BGR2GRAY)

    ## img_normalization

    decomp_img_1 = cv2.normalize(decomp_img_1, None, 0, 255, cv2.NORM_MINMAX)

    ## CLAHE

    decomp_img_1 = cv2.equalizeHist(decomp_img_1)
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(3,3)) 
    decomp_img_1= clahe.apply(decomp_img_1)          


    ## Threshold / value = img.mean()
    ret, mask = cv2.threshold(decomp_img_1,
                            np.mean(decomp_img_1),
                            255,
                            cv2.THRESH_BINARY) 

    ## Extract object / same value pixels

    contours, hierarchy = cv2.findContours(mask, 
                                            cv2.RETR_EXTERNAL, # only outline
                                            cv2.CHAIN_APPROX_SIMPLE) # Contour vertex coordinate


    ## drawing Contours

    cv2.drawContours(mask, contours, -1, (255,255,255), -1) # -1: 모든 컨트어 표시 /color/ fill


        
    ######## Decomposing_stage_2 / [ Brightness_Empahsis ] ########



    ## Empahsis
    decomp_img_2 = rotated_img.copy()
    if bright_ness(decomp_img_2) > 0.8:
        decomp_img_2 = np.clip(decomp_img_2 - 80., 0, 255).astype(np.uint8)
    elif bright_ness(decomp_img_2) > 0.75:
        decomp_img_2 = np.clip(decomp_img_2 - 50., 0, 255).astype(np.uint8)
    elif bright_ness(decomp_img_2) > 0.65:
        decomp_img_2 = np.clip(decomp_img_2 - 30., 0, 255).astype(np.uint8)
    else: decomp_img_2 = np.clip(decomp_img_2 - 10., 0, 255).astype(np.uint8)


    ## Morphology
    k2 = cv2.getStructuringElement(cv2.MORPH_CROSS,(b,b))
    decomp_img_2 = cv2.morphologyEx(decomp_img_2, cv2.MORPH_TOPHAT, k2)

    ## contrast
    decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_BGR2RGB)
    decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_BGR2GRAY)

    if decomp_img_2.mean() <= 15:
        low = decomp_img_2.mean() * 3.2
        high = decomp_img_2.mean() * 3.6
    elif decomp_img_2.mean() <= 20:
        low = decomp_img_2.mean() * 3
        high = decomp_img_2.mean() * 3.6
    else:
        low = decomp_img_2.mean() * 3
        high = decomp_img_2.mean() * 3.7

    decomp_img_2 = cv2.blur(decomp_img_2,(2,2))
    decomp_img_2 = contrast_roi(decomp_img_2, low, high)


    ######## Decomposing_Final_stage / [ Result ] ########

    ### Bone empahsis / bitwise (mask)

    ## Morphology

    ## Contours
    contours, hierarchy = cv2.findContours(decomp_img_2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(decomp_img_2, contours, -1, (255, 255, 255), -1)

    ## Bitwise (mask) / print white parts

    decomp_img_2 = cv2.bitwise_and(decomp_img_2, mask) 

    decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_GRAY2BGR)
    decomp_img_2 = cv2.blur(decomp_img_2,(2,2))

    bone_extraction = cv2.resize(decomp_img_2, (600, 800))

    return bone_extraction

### Preprocessing Running Code

In [ ]:
def Bone_extraction(path):
    try:
        global bone_path
        original_img = read_img(path)
        mask = make_mask(original_img)
        masked = cut_mask(original_img, mask)
        overlaied_img = overlay_contours(masked)
        rotated_img = img_rotation(overlaied_img)
        bone = Decomposing(rotated_img,60,55,50,25)
        bone_path = cv2.imwrite(path)
        return bone_path

    except:
        print('ERROR > Please check again' )

## Auto_processing Code ( No preprocessing Code)

In [ ]:
def input_info(NO, gender):
  global img_path, model_path, tjnet_path
  
  # Gender condition
  if gender == 1:
    ge = 'M'
    Gender_ = 'male'
  elif gender == 0:
    ge   = 'F'
    Gender_ = 'female'

  img_path =f'/content/drive/MyDrive/2차 프로젝트 원본 데이터/preprocessing_done/{Gender_}/{NO}_{ge}.jpg'
  model_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/yolov5'
  tjnet_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/TJM/tjnet24.h5'

  return [NO, gender, img_path, model_path, tjnet_path]

def predict_age(NO, gender):
  import pandas as pd
  df = pd.read_pickle('/content/drive/MyDrive/2차 프로젝트 원본 데이터/BA_all_.pkl')
  # df.columns
  ba_mean = df.BONE.mean()
  ba_std = df.BONE.std()
  return ba_mean, ba_std, df
  

def predict_zscore(gender, img_path, model_path,tjnet_path, show_crop, result_out):
  yolo_crop_img(gender, img_path, model_path, show_crop=show_crop,result_out = result_out)
  import numpy as np
  import tensorflow.keras as tf
  model = tf.models.load_model(tjnet_path, compile=False)
  # grobal X : X_ray 이미지의 yolo crop image 값
  y_predict = model.predict(X)
  global pred
  pred = y_predict[0][0]

def yolo_crop_img(gender, img_path, model_path, result_out=False, show_crop=False):
  import os
  import numpy as np
  os.chdir(model_path)
  import torch
  model = torch.load('./model.pt', map_location='cpu')
  result = model(img_path)
  global crops
  crops = result.crop(save=False)
  global X
  X = out_crop_img(crops, gender)
  global img
  img = np.squeeze(result.render())
  if show_crop: show_img(result)
  if result_out: return result

def show_img(result):
  import matplotlib.pyplot as plt
  import numpy as np
  %matplotlib inline
  plt.figure(figsize=(16,12))
  plt.imshow(np.squeeze(result.render()))
  plt.show()

def out_crop_img(crop, gender):
  import re
  import cv2
  import numpy as np

  gender = np.array(gender).reshape(1,1)

  for i in range(7):
    carpal = re.compile('CARPAL.')
    ip = re.compile('IP.')
    lmcp = re.compile('LMCP.')
    lpip = re.compile('LPIP.')
    mmcp = re.compile('MMCP.')
    mpip = re.compile('MPIP.')
    tmcp = re.compile('TMCP.')

    if carpal.search(crop[i]['label']):
      CARPAL_img = crop[i]['im']
      CARPAL_img = cv2.resize(CARPAL_img, (224,224),cv2.INTER_AREA)
      CARPAL_img = np.expand_dims(CARPAL_img, axis=0)

    if ip.search(crop[i]['label']):
      IP_img = crop[i]['im']
      IP_img = cv2.resize(IP_img, (75,75),cv2.INTER_AREA)
      IP_img = np.expand_dims(IP_img, axis=0)
      
    if lmcp.search(crop[i]['label']):
      LMCP_img = crop[i]['im']
      LMCP_img = cv2.resize(LMCP_img, (75,75),cv2.INTER_AREA)
      LMCP_img = np.expand_dims(LMCP_img, axis=0)

    if lpip.search(crop[i]['label']):
      LPIP_img = crop[i]['im']
      LPIP_img = cv2.resize(LPIP_img, (75,75),cv2.INTER_AREA)
      LPIP_img = np.expand_dims(LPIP_img, axis=0)
    
    if mmcp.search(crop[i]['label']):
      MMCP_img = crop[i]['im']
      MMCP_img = cv2.resize(MMCP_img, (75,75),cv2.INTER_AREA)
      MMCP_img = np.expand_dims(MMCP_img, axis=0)
    
    if mpip.search(crop[i]['label']):
      MPIP_img = crop[i]['im']
      MPIP_img = cv2.resize(MPIP_img, (75,75),cv2.INTER_AREA)
      MPIP_img = np.expand_dims(MPIP_img, axis=0)

    if tmcp.search(crop[i]['label']):
      TMCP_img = crop[i]['im']
      TMCP_img = cv2.resize(TMCP_img, (75,75),cv2.INTER_AREA)
      TMCP_img = np.expand_dims(TMCP_img, axis=0)

    else : continue

  
  return [CARPAL_img, LMCP_img, MMCP_img,TMCP_img, LPIP_img, MPIP_img, IP_img, gender]

### Auto Processing Running Code

In [ ]:
def final(NO,gender,show_crop=True,result_out=True):
    NO, gender, img_path, model_path, tjnet_path = input_info(NO,gender)
    bone, save_path = Bone_extraction(img_path)
    ba_mean, ba_std, df = predict_age(NO,gender)
    predict_zscore(gender, save_path, model_path,tjnet_path, show_crop, result_out)
    prediction_BA = (pred * ba_std + ba_mean)/12
    cond1 = df['GENDER'] == gender
    cond2 = df['NO'] == NO
    Actual_value = df[cond1&cond2]['BA_TOTAL'].values[0]
    Diff = abs( Actual_value - prediction_BA )*12
    print('Actual > {0} Year'.format( round(Actual_value,2)) )
    print('prediction_BA > {0} Year'.format( round(prediction_BA,2)))
    print('Different > {0} Month'.format(round(Diff,2)))
    return Diff

# Validation Acurracy Function

In [ ]:
def Total_pred_validation(NO,gender,show_crop=True,result_out=True):

########################### * Validation Accuracy Function *  ################################
##                                                                                          ##
##                                  * Functions *                                           ##
##        1. input_info : Information ( NO : index number, Gender : patient's gender )      ##
##        2. Bone_extraction : Extract bone from original x_ray img                         ##
##          2-1. read_img : Read img from path                                              ##
##          2-2. make_mask : Making mask for removing background                            ##
##          2-3. cut_mask : Cutting background based mask                                   ##
##          2-4. overlay_contours : Overlaying Contours                                     ##
##          2-5. img_rotation : Rotation                                                    ##
##          2-6. contrast_roi : Contrast for Decomposing                                    ##
##          2-7. bright_ness : Changing Brightness for Decomposing                          ##
##          2-8. Decomposing : Extracting Bone                                              ##
##        3. predict_age : Calculation Bone Age mean, std values in train_DataFrame         ##
##        4. predict_zscore : Using tjnet, predict Bone Age z_score value                   ##
##        5. yolo_crop_img : Cropping Img Using YoloV5                                      ##
##        6. show_img : Print Result Img                                                    ##
##        7. out_crop_img : Declaration Variable of crop images                             ##                     
##                                                                                          ##
##                                    * Notes *                                             ##
##           Several Time will be spent on loading Yolo model and TJnet model.              ##
##                                                                                          ##
##############################################################################################
    
    ## 1. input_info

    def input_info(NO, gender):
        global img_path, model_path, tjnet_path

        # Gender condition
        if gender == 1:
            ge = 'M'
            Gender_ = 'Male'
        elif gender == 0:
            ge   = 'F'
            Gender_ = 'Female'

        img_path =f'/content/drive/MyDrive/2차 프로젝트 원본 데이터/데이터 원본/image/{Gender_}/{NO}_{ge}.jpg'
        model_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/yolov5'
        tjnet_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/TJM/tjnet24.h5'

        return [NO, gender, img_path, model_path, tjnet_path]

    ## 2. Bone_extraction

    def Bone_extraction(img_path):

        # model import
        import numpy as np
        import cv2
        import matplotlib.pyplot as plt
        from sklearn.linear_model import LinearRegression
        import glob
        import math

        ## 2-1. read_img
        def read_img(path):
            original_img = cv2.imread(path)
            return original_img

        ## 2-2. make_mask
        def make_mask(original_img):
            # change to lab for making mask
            img_mask = original_img.copy()
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_RGB2BGR)
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2Lab)
            ## blur _02 
            # kernel_size = odds / value = img.mean()
            blur_k = int((img_mask.mean()*0.5)//2)*2+1
            img_mask = cv2.medianBlur(img_mask, blur_k)
            ## change to Grayscale for threshold
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_Lab2BGR)
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)
            ## binary / value = img.mean()
            if img_mask.mean() > 100 : 
                th = img_mask.mean()*0.94
            else : 
                th = img_mask.mean()
            ret, img_mask = cv2.threshold(img_mask, th, 255, cv2.THRESH_BINARY)
            ## mask based Max value of contours
            contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            max_cnt = max(contours, key=cv2.contourArea)
            mask = np.zeros(img_mask.shape, dtype=np.uint8)
            cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)
            ## Applying for dilation
            k = cv2.getStructuringElement(cv2.MORPH_RECT, (8,8))
            mask = cv2.dilate(mask,k)
            return mask

        ## 2-3. cut_mask
        def cut_mask(original_img, mask):
            ## copying
            img_for_cut = original_img.copy()
            ## H/W
            height, width = img_for_cut.shape[:2]
            ## mask
            mask_list = mask.tolist()    
            for y in range(int(height*0.05),height):
                if max(mask[y,int(width*0.3):int(width*0.7)]) > 0:
                    start_y = y-int(height*0.05)
                    break            
            for x in range(int(width*0.05),width):
                if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
                    start_x = x-int(width*0.05)
                    break           
            for x in range(int(width*0.95),-1,-1):
                if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
                    end_x = x+int(width*0.05)
                    break               
            cut_index = 0
            if mask_list[height-1][-1] == 255 or mask_list[height-1][0] == 255:
                for n in reversed(range(height)):
                    if mask_list[n][0] == 0 or mask_list[n][-1] == 0:
                        cut_index = n
                        break            
            if cut_index == 0:
                cut_index = height
            ## converting color
            img_for_cut = cv2.cvtColor(img_for_cut, cv2.COLOR_BGR2GRAY) 
            img_for_cut = img_for_cut[start_y:(cut_index-1),start_x:end_x]
            mask = mask[start_y:(cut_index-1),start_x:end_x]
            ## remove background
            masked = cv2.bitwise_and(img_for_cut, mask)
            return masked
        
        ## 2-4. overlay_contours
        def overlay_contours(masked):
            copied_img = masked.copy()
            ## smoothing
            gray_smooth = cv2.bilateralFilter(copied_img, 7,60,60)
            ## contours
            outline = cv2.Canny(gray_smooth,50,50)
            ## overlay
            overlaied_img = cv2.add(masked,outline)
            return overlaied_img

        ## 2-5. img_rotation
        def img_rotation(overlaied_img):
            ## copying img
            before_rot_img = overlaied_img.copy()
            h, w = before_rot_img.shape[:2]
            before_rot_img = cv2.cvtColor(before_rot_img, cv2.COLOR_RGB2BGR)
            gray = cv2.cvtColor(before_rot_img, cv2.COLOR_BGR2GRAY)
            ret, th = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
            th_li = th.tolist()
            ## Rotation stage 01
            # lower = first black spot
            for i in reversed(range(h)):
                if th_li[i][0] == 0 and th_li[i][-1] == 0:
                    lower = i
                    break
            # lower = condition ; bottom = lower / img * 0.95
            if lower == h - 1:
                lower = int(h*0.9)
            # upper = condition ; lower + lower * 0.05
            slice5 = int(len(th)*0.05)
            upper = lower - slice5
            # x, y = between upper and lower (5%) / wrist center
            x,y = [],[]
            for i in range(slice5):
                cnt = th_li[i + upper].count(255)
                index = th_li[i + upper].index(255)
                x.append([i+upper])
                y.append([int((index*2 + cnt - 1)/2)])
            # x, y / draw regression line
            model = LinearRegression()
            model.fit(X=x,y=y)
            ####################################################
            ## Rotation stage 02        
            angle = math.atan2(h - 0, int(model.predict([[h]])) - int(model.predict([[0]])))*180/math.pi
            M = cv2.getRotationMatrix2D((w/2,h/2), angle-90, 1)
            rotate = cv2.warpAffine(before_rot_img, M, (w, h))
            # Cutting img (rotated img)
            for i in range(len(th[-1])):
                if th[-1][i] == 255:
                    start_x = i
                    break
            for i in range(len(th[-1])):
                if th[-1][i] == 255:
                    end_x = i                   
            s_point = h - int((int(model.predict([[h]])-start_x)) * math.tan(math.pi*((90-angle)/180)))
            e_point = h - int((end_x - int(model.predict([[h]]))) * math.tan(math.pi*((angle-90)/180)))
            point = max(s_point, e_point)
            rotated_img = rotate[:point]
            return rotated_img

        ## 2-6. contrast_roi
        def contrast_roi(img, low, high):
            ## height, width
            h, w = img.shape
            img_ = np.zeros(img.shape, dtype=np.uint8)
            for y in range(h):
                for x in range(w):
                    temp = int((255 / (high - low)) * (img[y][x] - low))
                    if temp > 255:
                        img_[y][x] = 255
                    elif temp < 0:
                        img_[y][x] = 0
                    else:
                        img_[y][x] = temp
            return img_

        ## 2-7. bright_ness
        def bright_ness(img):
            ## columns, rows
            cols, rows = img.shape[:2]
            brightness = np.sum(img) / (255 * cols * rows)
            return brightness

        ## 2-8. Decomposing
        def Decomposing(rotated_img,a,b,d,e):
            ######## Decomposing_stage_1 / [ Contours , Mask ] ########
            decomp_img_1 = rotated_img.copy()
            ## Adjusting brighness
            if bright_ness(decomp_img_1) > 0.8:
                decomp_img_1 = np.clip(decomp_img_1 - 80., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_1) > 0.75:
                decomp_img_1 = np.clip(decomp_img_1 - 50., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_1) > 0.65:
                decomp_img_1 = np.clip(decomp_img_1 - 30., 0, 255).astype(np.uint8)
            else: decomp_img_1 = np.clip(decomp_img_1 - 10., 0, 255).astype(np.uint8)
            ## change to Lab
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_RGB2BGR)
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_BGR2Lab)
            ## Morphology
            k = cv2.getStructuringElement(cv2.MORPH_CROSS, (a, a))
            decomp_img_1 = cv2.morphologyEx(decomp_img_1, cv2.MORPH_TOPHAT, k) # Emphasis
            ## Filter
            decomp_img_1 = cv2.bilateralFilter(decomp_img_1,-1, d, e)
            ## Lab to gray for binary
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_Lab2BGR)
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_BGR2GRAY)
            ## img_normalization
            decomp_img_1 = cv2.normalize(decomp_img_1, None, 0, 255, cv2.NORM_MINMAX)
            ## CLAHE
            decomp_img_1 = cv2.equalizeHist(decomp_img_1)
            clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(3,3)) 
            decomp_img_1= clahe.apply(decomp_img_1)          
            ## Threshold / value = img.mean()
            ret, mask = cv2.threshold(decomp_img_1,
                                    np.mean(decomp_img_1),
                                    255,
                                    cv2.THRESH_BINARY) 
            ## Extract object / same value pixels
            contours, hierarchy = cv2.findContours(mask, 
                                                    cv2.RETR_EXTERNAL, # only outline
                                                    cv2.CHAIN_APPROX_SIMPLE) # Contour vertex coordinate
            ## drawing Contours
            cv2.drawContours(mask, contours, -1, (255,255,255), -1) # -1: 모든 컨트어 표시 /color/ fill             
            ######## Decomposing_stage_2 / [ Brightness_Empahsis ] ########
            ## Empahsis
            decomp_img_2 = rotated_img.copy()
            if bright_ness(decomp_img_2) > 0.8:
                decomp_img_2 = np.clip(decomp_img_2 - 80., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_2) > 0.75:
                decomp_img_2 = np.clip(decomp_img_2 - 50., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_2) > 0.65:
                decomp_img_2 = np.clip(decomp_img_2 - 30., 0, 255).astype(np.uint8)
            else: decomp_img_2 = np.clip(decomp_img_2 - 10., 0, 255).astype(np.uint8)
            ## Morphology
            k2 = cv2.getStructuringElement(cv2.MORPH_CROSS,(b,b))
            decomp_img_2 = cv2.morphologyEx(decomp_img_2, cv2.MORPH_TOPHAT, k2)
            ## contrast
            decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_BGR2RGB)
            decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_BGR2GRAY)
            if decomp_img_2.mean() <= 15:
                low = decomp_img_2.mean() * 3.2
                high = decomp_img_2.mean() * 3.6
            elif decomp_img_2.mean() <= 20:
                low = decomp_img_2.mean() * 3
                high = decomp_img_2.mean() * 3.6
            else:
                low = decomp_img_2.mean() * 3
                high = decomp_img_2.mean() * 3.7
            decomp_img_2 = cv2.blur(decomp_img_2,(2,2))
            decomp_img_2 = contrast_roi(decomp_img_2, low, high)
            ######## Decomposing_Final_stage / [ Result ] ########
            ### Bone empahsis / bitwise (mask)
            ## Morphology
            ## Contours
            contours, hierarchy = cv2.findContours(decomp_img_2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(decomp_img_2, contours, -1, (255, 255, 255), -1)
            ## Bitwise (mask) / print white parts
            decomp_img_2 = cv2.bitwise_and(decomp_img_2, mask) 
            decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_GRAY2BGR)
            decomp_img_2 = cv2.blur(decomp_img_2,(2,2))
            bone_extraction = cv2.resize(decomp_img_2, (600, 800))
            return bone_extraction

        ## Bone_extraction (Running code)
        try:
            global bone
            original_img = read_img(img_path)
            mask = make_mask(original_img)
            masked = cut_mask(original_img, mask)
            # overlaied_img = overlay_contours(masked)
            rotated_img = img_rotation(masked)
            bone = Decomposing(rotated_img,60,55,50,25)
            
            # Save extraction img
            global save_path
            import re
            regex = re.compile('[^.+]+')
            test = regex.findall(img_path)
            save_path = test[0] + '_extraction' + '.jpg'
            cv2.imwrite(save_path, bone)
            return bone, save_path

        except:
            print('ERROR > Please check again' )


    ## 3. predict_age

    def predict_age(NO, gender):
        import pandas as pd
        df = pd.read_pickle('/content/drive/MyDrive/2차 프로젝트 원본 데이터/BA_all_.pkl')
        ba_mean = df.BONE.mean()
        ba_std = df.BONE.std()
        return ba_mean, ba_std, df

    ## 4. predict_zscore

    def predict_zscore(gender, save_path, model_path,tjnet_path, show_crop, result_out):

        # Function connection line ; yolo_crop_img(4)
        yolo_crop_img(gender, save_path, model_path, show_crop=show_crop,result_out = result_out)

        import numpy as np
        import tensorflow.keras as tf

        model = tf.models.load_model(tjnet_path, compile=False)

        # grobal X : X_ray 이미지의 yolo crop image 값

        y_predict = model.predict(X)
        global pred
        pred = y_predict[0][0] 

    ## 5. yolo_crop_img

    def yolo_crop_img(gender, save_path, model_path, result_out=False, show_crop=False):
        import os
        import numpy as np

        os.chdir(model_path)
        import torch
    
        model = torch.load('./model.pt', map_location='cpu')
        result = model(save_path)
        global crops
        crops = result.crop(save=False)
        global X
        X = out_crop_img(crops, gender)
        global img
        img = np.squeeze(result.render())
        if show_crop: show_img(result)
        if result_out: return result

    ## 6. show_img

    def show_img(result):
        import matplotlib.pyplot as plt
        import numpy as np
        %matplotlib inline
        plt.figure(figsize=(16,12))
        plt.imshow(np.squeeze(result.render()))
        plt.show()

    ## 7. out_crop_img

    def out_crop_img(crop, gender):
        import re
        import cv2
        import numpy as np

        gender = np.array(gender).reshape(1,1)

        for i in range(7):
            carpal = re.compile('CARPAL.')
            ip = re.compile('IP.')
            lmcp = re.compile('LMCP.')
            lpip = re.compile('LPIP.')
            mmcp = re.compile('MMCP.')
            mpip = re.compile('MPIP.')
            tmcp = re.compile('TMCP.')

            if carpal.search(crop[i]['label']):
                CARPAL_img = crop[i]['im']
                CARPAL_img = cv2.resize(CARPAL_img, (224,224),cv2.INTER_AREA)
                CARPAL_img = np.expand_dims(CARPAL_img, axis=0)

            if ip.search(crop[i]['label']):
                IP_img = crop[i]['im']
                IP_img = cv2.resize(IP_img, (75,75),cv2.INTER_AREA)
                IP_img = np.expand_dims(IP_img, axis=0)
                
            if lmcp.search(crop[i]['label']):
                LMCP_img = crop[i]['im']
                LMCP_img = cv2.resize(LMCP_img, (75,75),cv2.INTER_AREA)
                LMCP_img = np.expand_dims(LMCP_img, axis=0)

            if lpip.search(crop[i]['label']):
                LPIP_img = crop[i]['im']
                LPIP_img = cv2.resize(LPIP_img, (75,75),cv2.INTER_AREA)
                LPIP_img = np.expand_dims(LPIP_img, axis=0)
            
            if mmcp.search(crop[i]['label']):
                MMCP_img = crop[i]['im']
                MMCP_img = cv2.resize(MMCP_img, (75,75),cv2.INTER_AREA)
                MMCP_img = np.expand_dims(MMCP_img, axis=0)
                
            if mpip.search(crop[i]['label']):
                MPIP_img = crop[i]['im']
                MPIP_img = cv2.resize(MPIP_img, (75,75),cv2.INTER_AREA)
                MPIP_img = np.expand_dims(MPIP_img, axis=0)

            if tmcp.search(crop[i]['label']):
                TMCP_img = crop[i]['im']
                TMCP_img = cv2.resize(TMCP_img, (75,75),cv2.INTER_AREA)
                TMCP_img = np.expand_dims(TMCP_img, axis=0)

            else : continue

        return [CARPAL_img, LMCP_img, MMCP_img,TMCP_img, LPIP_img, MPIP_img, IP_img, gender]

#### All Functions line ####

    NO, gender, img_path, model_path, tjnet_path = input_info(NO,gender)
    bone, save_path = Bone_extraction(img_path)
    ba_mean, ba_std, df = predict_age(NO,gender)
    predict_zscore(gender, save_path, model_path,tjnet_path, show_crop, result_out)
    prediction_BA = (pred * ba_std + ba_mean)/12
    cond1 = df['GENDER'] == gender
    cond2 = df['NO'] == NO
    Actual_value = df[cond1&cond2]['BA_TOTAL'].values[0]
    Diff = abs( Actual_value - prediction_BA )*12
    print('Actual > {0} Year'.format( round(Actual_value,2)) )
    print('prediction_BA > {0} Year'.format( round(prediction_BA,2)))
    print('Different > {0} Month'.format(round(Diff,2)))
    return Diff

### Validation Accuracy Code _ Test code

In [ ]:
Total_pred_validation(1,0,True,True)
# orginal_img, no=1, gender = 0(female) , showing img = True, Return img data = True
# no apply canny_edge
# img_path =f'/content/drive/MyDrive/2차 프로젝트 원본 데이터/preprocessing_done/{Gender_}/{NO}_{ge}.jpg'

### Blend_Altman

##### copy images

In [ ]:
# from pickle5 import pickle
# import shutil
# import os
# with open('/content/drive/MyDrive/2차 프로젝트 원본 데이터/val.pkl','rb') as file:
#     val = pickle.load(file)
# path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/데이터 원본/image/'
# No_list =val['NO'].tolist()

# path_list = []
# for i,NO in enumerate(No_list):
#     if val[val['NO']==NO]['GENDER'][i] == 0:
#         Female = 'Female/'
#         F_short = 'F'
#         img_path = path + Female + str(NO) + '_' + F_short + '.jpg'
#     elif val[val['NO']==NO]['GENDER'][i] == 1:
#         Male = 'Male/'
#         M_short = 'M'
#         img_path = path + Male + str(NO) + '_' + M_short + '.jpg'
#     path_list.append(img_path)



# last_list = []
# for i,copy_file in enumerate(path_list):
#     try:
#         destination = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/B_A_img_data/'
#         shutil.copy(copy_file, destination)
    
#     except:
#         last_list.append(copy_file)

# import re
# regex = re.compile('[^.+]+')
# regex2 = re.compile(r'\d\w+')
# for i, copy_file in enumerate(last_list):
#     try:
#         test = regex.findall(copy_file)
#         text = regex2.findall(test[0])
#         path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/데이터 원본/image/Male/' +'0' + text[1] + '.jpg'
#         destination = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/B_A_img_data/'
#         shutil.copy(path, destination)
#     except :
#         path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/데이터 원본/image/Male/' +'00' + text[1] + '.jpg'
#         destination = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/B_A_img_data/'
#         shutil.copy(path, destination)

##### Prediction

In [ ]:
# Test
## For Loop
import pandas as pd
import re
regex = re.compile('[^.+]+')
regex2 = re.compile(r'\d+')

import os
path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/B_A_img_data/'
dir_list = os.listdir(path)
# print(dir_list)
path_list = []
for p in dir_list:
    path_list.append(path+p)


prediction = []
nope = []
from tqdm import tqdm
for path in tqdm(path_list):
    try:
        if '_F' in path:
            gender = 0
        elif '_M' in path:
            gender = 1

        img_path = path 

        df = pd.read_pickle('/content/drive/MyDrive/2차 프로젝트 원본 데이터/BA_all_.pkl')    
        test = regex.findall(path)
        text = regex2.findall(test[0]) 
        # A = df[ (df['NO']==int(text[1])) & (df['GENDER']==gender) ][['NO','BA_1','BA_2','BA_TOTAL']]
        print()
        print('INFO > ', path)
        print('NO > ', text[1], 'Gender > ', gender)
        # display('Real_age_in_df > ', A)
        Total_pred_gui(gender)
        # print('prediction_BA > {0} Year'.format( round(prediction_BA,2)))

        pred = round(prediction_BA,2) 
        Result = str(text[1])+','+str(gender)+','+str(pred)
        prediction.append(Result)
    except:
        nope.append(path)
        print('안됨')
        print(path)

display(prediction)
nope

##### DataFrame for Comparing

In [ ]:
import pandas as pd
validation_result = pd.read_csv('/content/drive/MyDrive/2차 프로젝트 원본 데이터/validation_result.csv',index_col=0)
validation_result = validation_result.rename(columns={'0': 'value'})
validation_result = validation_result['value'].str.split(',', expand = True)
validation_result = validation_result.rename(columns = {0 : 'NO', 1: 'GENDER', 2: 'PREDICTION'})

val_df = pd.read_pickle('/content/drive/MyDrive/2차 프로젝트 원본 데이터/val.pkl')
val_df = val_df[['NO','GENDER','DIG_AGE','BA_1','BA_2','BA_TOTAL']]

validation_result['NO'] = validation_result['NO'].astype(int)
validation_result['GENDER'] = validation_result['GENDER'].astype(int)
validation_result['PREDICTION'] = validation_result['PREDICTION'].astype(float)
Compare_df = pd.merge(val_df, validation_result, on=['NO','GENDER'],how='left')
Compare_df.info()

# ERROR 제외
Compare_df[Compare_df['NO']==490] # 15
Compare_df[Compare_df['NO']==196] # 41
Compare_df[Compare_df['NO']==217] # 45
Compare_df=Compare_df.drop([15,41,45])

In [ ]:
Compare_df['1-PRED'] = Compare_df['BA_1'] - Compare_df['PREDICTION']
Compare_df['2-PRED'] = Compare_df['BA_2'] - Compare_df['PREDICTION']
Compare_df['TOTAL-PRED'] = Compare_df['BA_TOTAL'] - Compare_df['PREDICTION']

# # 저장
# Compare_df.to_csv('/content/drive/MyDrive/2차 프로젝트 원본 데이터/Comparing_df.csv')
Compare_df

#### BLEND_ALTMAN

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(Compare_df['PREDICTION'],
                           Compare_df['BA_1'],
                           ax = ax)
plt.title('BA_1 vs PREDICTION')
plt.show()

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(Compare_df['PREDICTION'],
                           Compare_df['BA_2'],
                           ax = ax)
plt.title('BA_2 vs PREDICTION')
plt.show()

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(Compare_df['PREDICTION'],
                           Compare_df['BA_TOTAL'],
                           ax = ax)
plt.title('BA_TOTAL vs PREDICTION')
plt.show()

# GUI code
- show_crop / result_out no function
- show_img function can operation separately

In [ ]:
# img_path = uploading img path
# gender = inputing info
def Total_pred_gui(gender):

###########################  * Prediction Function for GUI *  ################################
##                                                                                          ##
##                                  * Functions *                                           ##
##        1. input_info Information ( NO : index number, Gender : patient's gender )        ##
##        2. Bone_extraction : Extract bone from original x_ray img                         ##
##          2-1. read_img : Read img from path                                              ##
##          2-2. make_mask : Making mask for removing background                            ##
##          2-3. cut_mask : Cutting background based mask                                   ##
##          2-4. overlay_contours : Overlaying Contours                                     ##
##          2-5. img_rotation : Rotation                                                    ##
##          2-6. contrast_roi : Contrast for Decomposing                                    ##
##          2-7. bright_ness : Changing Brightness for Decomposing                          ##
##          2-8. Decomposing : Extracting Bone                                              ##
##        3. predict_zscore : Using tjnet, predict Bone Age z_score value                   ##
##        4. yolo_crop_img : Cropping Img Using YoloV5                                      ##
##        5. show_img : Print Result Img                                                    ##
##        6. out_crop_img : Declaration Variable of crop images                             ##                     
##                                                                                          ##
##                                    * Notes *                                             ##
##        1. Several Time will be spent on loading Yolo model and TJnet model.              ##
##        2. BA_mean value = 115.41626213592232                                             ##
##        3. BA_std value = 48.02950411666953                                               ##
##        4. BA mean, std values are Calculated by train DataFrame                          ##
##                                                                                          ##
##############################################################################################

    ## 1. input_info

    def input_info(gender):
        global img_path, model_path, tjnet_path

        gender = gender
        img_path = img_path
        model_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/yolov5'
        tjnet_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/TJM/tjnet24.h5'

        return [gender, img_path, model_path, tjnet_path]

    ## 2. Bone_extraction

    def Bone_extraction(img_path):

        # model import
        import numpy as np
        import cv2
        import matplotlib.pyplot as plt
        from sklearn.linear_model import LinearRegression
        import glob
        import math

        ## 2-1. read_img
        def read_img(path):
            original_img = cv2.imread(path)
            return original_img

        ## 2-2. make_mask
        def make_mask(original_img):
            # change to lab for making mask
            img_mask = original_img.copy()
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_RGB2BGR)
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2Lab)
            ## blur _02 
            # kernel_size = odds / value = img.mean()
            blur_k = int((img_mask.mean()*0.5)//2)*2+1
            img_mask = cv2.medianBlur(img_mask, blur_k)
            ## change to Grayscale for threshold
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_Lab2BGR)
            img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)
            ## binary / value = img.mean()
            if img_mask.mean() > 100 : 
                th = img_mask.mean()*0.94
            else : 
                th = img_mask.mean()
            ret, img_mask = cv2.threshold(img_mask, th, 255, cv2.THRESH_BINARY)
            ## mask based Max value of contours
            contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            max_cnt = max(contours, key=cv2.contourArea)
            mask = np.zeros(img_mask.shape, dtype=np.uint8)
            cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)
            ## Applying for dilation
            k = cv2.getStructuringElement(cv2.MORPH_RECT, (8,8))
            mask = cv2.dilate(mask,k)
            return mask

        ## 2-3. cut_mask
        def cut_mask(original_img, mask):
            ## copying
            img_for_cut = original_img.copy()
            ## H/W
            height, width = img_for_cut.shape[:2]
            ## mask
            mask_list = mask.tolist()    
            for y in range(int(height*0.05),height):
                if max(mask[y,int(width*0.3):int(width*0.7)]) > 0:
                    start_y = y-int(height*0.05)
                    break            
            for x in range(int(width*0.05),width):
                if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
                    start_x = x-int(width*0.05)
                    break           
            for x in range(int(width*0.95),-1,-1):
                if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
                    end_x = x+int(width*0.05)
                    break               
            cut_index = 0
            if mask_list[height-1][-1] == 255 or mask_list[height-1][0] == 255:
                for n in reversed(range(height)):
                    if mask_list[n][0] == 0 or mask_list[n][-1] == 0:
                        cut_index = n
                        break            
            if cut_index == 0:
                cut_index = height
            ## converting color
            img_for_cut = cv2.cvtColor(img_for_cut, cv2.COLOR_BGR2GRAY) 
            img_for_cut = img_for_cut[start_y:(cut_index-1),start_x:end_x]
            mask = mask[start_y:(cut_index-1),start_x:end_x]
            ## remove background
            masked = cv2.bitwise_and(img_for_cut, mask)
            return masked
        
        ## 2-4. overlay_contours
        def overlay_contours(masked):
            copied_img = masked.copy()
            ## smoothing
            gray_smooth = cv2.bilateralFilter(copied_img, 7,60,60)
            ## contours
            outline = cv2.Canny(gray_smooth,50,50)
            ## overlay
            overlaied_img = cv2.add(masked,outline)
            return overlaied_img

        ## 2-5. img_rotation
        def img_rotation(overlaied_img):
            ## copying img
            before_rot_img = overlaied_img.copy()
            h, w = before_rot_img.shape[:2]
            before_rot_img = cv2.cvtColor(before_rot_img, cv2.COLOR_RGB2BGR)
            gray = cv2.cvtColor(before_rot_img, cv2.COLOR_BGR2GRAY)
            ret, th = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
            th_li = th.tolist()
            ## Rotation stage 01
            # lower = first black spot
            for i in reversed(range(h)):
                if th_li[i][0] == 0 and th_li[i][-1] == 0:
                    lower = i
                    break
            # lower = condition ; bottom = lower / img * 0.95
            if lower == h - 1:
                lower = int(h*0.9)
            # upper = condition ; lower + lower * 0.05
            slice5 = int(len(th)*0.05)
            upper = lower - slice5
            # x, y = between upper and lower (5%) / wrist center
            x,y = [],[]
            for i in range(slice5):
                cnt = th_li[i + upper].count(255)
                index = th_li[i + upper].index(255)
                x.append([i+upper])
                y.append([int((index*2 + cnt - 1)/2)])
            # x, y / draw regression line
            model = LinearRegression()
            model.fit(X=x,y=y)
            ####################################################
            ## Rotation stage 02        
            angle = math.atan2(h - 0, int(model.predict([[h]])) - int(model.predict([[0]])))*180/math.pi
            M = cv2.getRotationMatrix2D((w/2,h/2), angle-90, 1)
            rotate = cv2.warpAffine(before_rot_img, M, (w, h))
            # Cutting img (rotated img)
            for i in range(len(th[-1])):
                if th[-1][i] == 255:
                    start_x = i
                    break
            for i in range(len(th[-1])):
                if th[-1][i] == 255:
                    end_x = i                   
            s_point = h - int((int(model.predict([[h]])-start_x)) * math.tan(math.pi*((90-angle)/180)))
            e_point = h - int((end_x - int(model.predict([[h]]))) * math.tan(math.pi*((angle-90)/180)))
            point = max(s_point, e_point)
            rotated_img = rotate[:point]
            return rotated_img

        ## 2-6. contrast_roi
        def contrast_roi(img, low, high):
            ## height, width
            h, w = img.shape
            img_ = np.zeros(img.shape, dtype=np.uint8)
            for y in range(h):
                for x in range(w):
                    temp = int((255 / (high - low)) * (img[y][x] - low))
                    if temp > 255:
                        img_[y][x] = 255
                    elif temp < 0:
                        img_[y][x] = 0
                    else:
                        img_[y][x] = temp
            return img_

        ## 2-7. bright_ness
        def bright_ness(img):
            ## columns, rows
            cols, rows = img.shape[:2]
            brightness = np.sum(img) / (255 * cols * rows)
            return brightness

        ## 2-8. Decomposing
        def Decomposing(rotated_img,a,b,d,e):
            ######## Decomposing_stage_1 / [ Contours , Mask ] ########
            decomp_img_1 = rotated_img.copy()
            ## Adjusting brighness
            if bright_ness(decomp_img_1) > 0.8:
                decomp_img_1 = np.clip(decomp_img_1 - 80., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_1) > 0.75:
                decomp_img_1 = np.clip(decomp_img_1 - 50., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_1) > 0.65:
                decomp_img_1 = np.clip(decomp_img_1 - 30., 0, 255).astype(np.uint8)
            else: decomp_img_1 = np.clip(decomp_img_1 - 10., 0, 255).astype(np.uint8)
            ## change to Lab
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_RGB2BGR)
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_BGR2Lab)
            ## Morphology
            k = cv2.getStructuringElement(cv2.MORPH_CROSS, (a, a))
            decomp_img_1 = cv2.morphologyEx(decomp_img_1, cv2.MORPH_TOPHAT, k) # Emphasis
            ## Filter
            decomp_img_1 = cv2.bilateralFilter(decomp_img_1,-1, d, e)
            ## Lab to gray for binary
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_Lab2BGR)
            decomp_img_1 = cv2.cvtColor(decomp_img_1, cv2.COLOR_BGR2GRAY)
            ## img_normalization
            decomp_img_1 = cv2.normalize(decomp_img_1, None, 0, 255, cv2.NORM_MINMAX)
            ## CLAHE
            decomp_img_1 = cv2.equalizeHist(decomp_img_1)
            clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(3,3)) 
            decomp_img_1= clahe.apply(decomp_img_1)          
            ## Threshold / value = img.mean()
            ret, mask = cv2.threshold(decomp_img_1,
                                    np.mean(decomp_img_1),
                                    255,
                                    cv2.THRESH_BINARY) 
            ## Extract object / same value pixels
            contours, hierarchy = cv2.findContours(mask, 
                                                    cv2.RETR_EXTERNAL, # only outline
                                                    cv2.CHAIN_APPROX_SIMPLE) # Contour vertex coordinate
            ## drawing Contours
            cv2.drawContours(mask, contours, -1, (255,255,255), -1) # -1: 모든 컨트어 표시 /color/ fill             
            ######## Decomposing_stage_2 / [ Brightness_Empahsis ] ########
            ## Empahsis
            decomp_img_2 = rotated_img.copy()
            if bright_ness(decomp_img_2) > 0.8:
                decomp_img_2 = np.clip(decomp_img_2 - 80., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_2) > 0.75:
                decomp_img_2 = np.clip(decomp_img_2 - 50., 0, 255).astype(np.uint8)
            elif bright_ness(decomp_img_2) > 0.65:
                decomp_img_2 = np.clip(decomp_img_2 - 30., 0, 255).astype(np.uint8)
            else: decomp_img_2 = np.clip(decomp_img_2 - 10., 0, 255).astype(np.uint8)
            ## Morphology
            k2 = cv2.getStructuringElement(cv2.MORPH_CROSS,(b,b))
            decomp_img_2 = cv2.morphologyEx(decomp_img_2, cv2.MORPH_TOPHAT, k2)
            ## contrast
            decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_BGR2RGB)
            decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_BGR2GRAY)
            if decomp_img_2.mean() <= 15:
                low = decomp_img_2.mean() * 3.2
                high = decomp_img_2.mean() * 3.6
            elif decomp_img_2.mean() <= 20:
                low = decomp_img_2.mean() * 3
                high = decomp_img_2.mean() * 3.6
            else:
                low = decomp_img_2.mean() * 3
                high = decomp_img_2.mean() * 3.7
            decomp_img_2 = cv2.blur(decomp_img_2,(2,2))
            decomp_img_2 = contrast_roi(decomp_img_2, low, high)
            ######## Decomposing_Final_stage / [ Result ] ########
            ### Bone empahsis / bitwise (mask)
            ## Morphology
            ## Contours
            contours, hierarchy = cv2.findContours(decomp_img_2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(decomp_img_2, contours, -1, (255, 255, 255), -1)
            ## Bitwise (mask) / print white parts
            decomp_img_2 = cv2.bitwise_and(decomp_img_2, mask) 
            decomp_img_2 = cv2.cvtColor(decomp_img_2, cv2.COLOR_GRAY2BGR)
            decomp_img_2 = cv2.blur(decomp_img_2,(2,2))
            bone_extraction = cv2.resize(decomp_img_2, (600, 800))
            return bone_extraction

        ## Bone_extraction (Running code)
        try:
            global bone
            original_img = read_img(img_path)
            mask = make_mask(original_img)
            masked = cut_mask(original_img, mask)
            # overlaied_img = overlay_contours(masked)
            rotated_img = img_rotation(masked)
            bone = Decomposing(rotated_img,60,55,50,25)
            
            # Save extraction img
            global save_path
            import re
            regex = re.compile('[^.+]+')
            test = regex.findall(img_path)
            save_path = test[0] + '_extraction' + '.jpg'
            cv2.imwrite(save_path, bone)
            return bone, save_path

        except:
            print('ERROR > Please check again' )

    ## 3. predict_zscore

    def predict_zscore(gender,save_path, model_path,tjnet_path):

        # Function connection line ; yolo_crop_img(4)
        yolo_crop_img(gender, save_path, model_path)

        import  numpy as np
        import tensorflow.keras as tf

        model = tf.models.load_model(tjnet_path, compile=False)

        # grobal X : X_ray 이미지의 yolo crop image 값

        y_predict = model.predict(X)
        global pred
        pred = y_predict[0][0] 

    ## 4. yolo_crop_img


    def yolo_crop_img(gender, save_path,model_path):
        import os
        import numpy as np
        global result
        os.chdir(model_path)
        import torch
    
        model = torch.load('./model.pt', map_location='cpu')
        result = model(save_path)
        global crops
        crops = result.crop(save=False)
        global X
        X = out_crop_img(crops, gender)
        global img
        img = np.squeeze(result.render())
        return result


    ## 5. show_img
    global show_img
    def show_img(result):
        import matplotlib.pyplot as plt
        import numpy as np
        %matplotlib inline
        plt.figure(figsize=(16,12))
        plt.imshow(np.squeeze(result.render()))
        plt.show()

    ## 6. out_crop_img

    def out_crop_img(crop, gender):
        import re
        import cv2
        import numpy as np

        gender = np.array(gender).reshape(1,1)

        for i in range(7):
            carpal = re.compile('CARPAL.')
            ip = re.compile('IP.')
            lmcp = re.compile('LMCP.')
            lpip = re.compile('LPIP.')
            mmcp = re.compile('MMCP.')
            mpip = re.compile('MPIP.')
            tmcp = re.compile('TMCP.')

            if carpal.search(crop[i]['label']):
                CARPAL_img = crop[i]['im']
                CARPAL_img = cv2.resize(CARPAL_img, (224,224),cv2.INTER_AREA)
                CARPAL_img = np.expand_dims(CARPAL_img, axis=0)

            if ip.search(crop[i]['label']):
                IP_img = crop[i]['im']
                IP_img = cv2.resize(IP_img, (75,75),cv2.INTER_AREA)
                IP_img = np.expand_dims(IP_img, axis=0)
                
            if lmcp.search(crop[i]['label']):
                LMCP_img = crop[i]['im']
                LMCP_img = cv2.resize(LMCP_img, (75,75),cv2.INTER_AREA)
                LMCP_img = np.expand_dims(LMCP_img, axis=0)

            if lpip.search(crop[i]['label']):
                LPIP_img = crop[i]['im']
                LPIP_img = cv2.resize(LPIP_img, (75,75),cv2.INTER_AREA)
                LPIP_img = np.expand_dims(LPIP_img, axis=0)
            
            if mmcp.search(crop[i]['label']):
                MMCP_img = crop[i]['im']
                MMCP_img = cv2.resize(MMCP_img, (75,75),cv2.INTER_AREA)
                MMCP_img = np.expand_dims(MMCP_img, axis=0)
                
            if mpip.search(crop[i]['label']):
                MPIP_img = crop[i]['im']
                MPIP_img = cv2.resize(MPIP_img, (75,75),cv2.INTER_AREA)
                MPIP_img = np.expand_dims(MPIP_img, axis=0)

            if tmcp.search(crop[i]['label']):
                TMCP_img = crop[i]['im']
                TMCP_img = cv2.resize(TMCP_img, (75,75),cv2.INTER_AREA)
                TMCP_img = np.expand_dims(TMCP_img, axis=0)

            else : continue

        return [CARPAL_img, LMCP_img, MMCP_img,TMCP_img, LPIP_img, MPIP_img, IP_img, gender]

#### All Functions line ####

    gender, img_path, model_path, tjnet_path = input_info(gender)
    bone, save_path = Bone_extraction(img_path)
    predict_zscore(gender, save_path, model_path,tjnet_path) #return pred
    BA_mean = 115.41626213592232                                            
    BA_std = 48.02950411666953         
    global prediction_BA
    prediction_BA = (pred * BA_std + BA_mean)/12
    return prediction_BA

### GUI Code _ Test code

In [ ]:
img_path ='/content/drive/MyDrive/2차 프로젝트 원본 데이터/데이터 원본/image/Female/1_F.jpg'
gender = 0
Total_pred_gui(gender)
print('prediction_BA > {0} Year'.format( round(prediction_BA,2)))
show_img(result)

In [ ]:
# Test
## For Loop
import pandas as pd
import re
regex = re.compile('[^.+]+')
regex2 = re.compile(r'\d+')

# A = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/Code_test_data/015_M.jpg'
# B = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/Code_test_data/252_F.jpg'
# C = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/Code_test_data/663_M.jpg'
# D = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/Code_test_data/401_F.jpg'

# path_list = [A,B,C,D]
path_list = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/B_A_img_data/490_F.jpg',


df = pd.read_pickle('/content/drive/MyDrive/2차 프로젝트 원본 데이터/BA_all_.pkl')

from tqdm import tqdm
for path in tqdm(path_list):

    if '_F' in path:
        gender = 0
    elif '_M' in path:
        gender = 1

    img_path = path 

    df = pd.read_pickle('/content/drive/MyDrive/2차 프로젝트 원본 데이터/BA_all_.pkl')    
    test = regex.findall(path)
    text = regex2.findall(test[0]) 
    A = df[ (df['NO']==int(text[1])) & (df['GENDER']==gender) ][['NO','BA_1','BA_2','BA_TOTAL']]

    print()
    print('gender > ' , gender)
    display('Real_age_in_df > ', A)
    Total_pred_gui(gender)
    print('prediction_BA > {0} Year'.format( round(prediction_BA,2)))
    show_img(result)   